In [9]:
from datetime import datetime
from openpyxl import load_workbook
wb = load_workbook('C:\\Users\\d.maryanovskiy\\Desktop\\БКГ - Влад.xlsm')
sew = wb['Питон']
w = [x for x in sew.values]
groups_my = ['5 цех', 'Воротники', 'Наволочка', 'Первая нав.', 'Подушки', 'Прилепина', 'Стельки', 'Трикотаж', 'Туторы', 'Упаковка']
groups_vlad = ['78', 'Аппликатор', 'Бандажи беж', 'Бандажи чёр', 'Косынки', 'Стяжка', 'ТРГ']
current_date = datetime.now().date()

In [16]:
sew_planning(w, int(input('Load:')), int(input('Days:')), input('Group:'))

Load:16800
Days:7
Group:5 цех
[-5, 5.226141494928378, 7.1627642646055945, 7.675558051382844, 8.27405789186237, 10.316594006953261, 11.389995164622622, 17.79541877879951, 19.42857142857143, 19.518275333771065, 20.52671755725191, 20.58562059752247, 20.727903921936576]
min_d - -5
66 man_load-0
66 Минимальный остаток--5
66 Запонение ячейки для минимального остатка. Мощностей хватеает. Кроя хватает.  10
66 man_load 80
64 Проверка того, что остатки достаточно большие, что бы заполнять
65 Проверка того, что остатки достаточно большие, что бы заполнять
67 Проверка того, что остатки достаточно большие, что бы заполнять
68 Проверка того, что остатки достаточно большие, что бы заполнять
69 Проверка того, что остатки достаточно большие, что бы заполнять
70 Проверка того, что остатки НЕдостаточно большие, что бы заполнять
70 Заполнение ячейки для НЕ минимального остатка. Мощностей хватает. Крой есть 10
70 man_load 160
71 Проверка того, что остатки достаточно большие, что бы заполнять
72 Проверка то

'Done'

In [17]:
plan_saving()

In [18]:
delete_rows_and_cols()

{'Воротники', 'Упаковка', '5 цех', 'Наволочка', 'Туторы', 'Трикотаж', 'Прилепина'}


In [11]:
def kft_min(row, days):
    return round(w[row][13]*days/w[row][32]+1)*w[row][32]
def kft (row, days):
    return round((days - w[row][21])*w[row][13]/w[row][32])*w[row][32]

In [12]:
# w - весь лист Пошива Excel
# l - строки в Пошиве, содержащие "5 цех"
# min_days - список минимальных дней по ограничителям
# min_d - значение из списка с минимальными остатками в днях
# man_load - счетчик загруженности производства
# l_amont_min - количество, которое добавим в план в строку с минимальным остатком в артикуле
# l_residual_days - остаток в днях в артикуле и размере после плана, по которому были минимальные остатки
# l_amount - количество, которое добавим в план в остальные строки в артикуле
# kft_min - функция для расчета плана для строки с минимальными остатками

In [13]:
def sew_planning(w, man_load_max, days, group):
    min_days = []
    man_load = 0
    count_numbers_of_man_load_cycles = 0
    for i in range(len(w)):
        if group == w[i][8]:
            min_days.append(w[i][20])
    min_days_sorted = list(sorted(set(min_days)))
    print(min_days_sorted)
    #lims_n = 0
    # Итерации для прохождения по списку lims
    for min_d in min_days_sorted:
        #lims_n = lims_sorted[n]
        print('min_d - '+str(min_d))       
    # Пробегаем по таблице в поиске мин остатка в днях
        for k in range(len(w)):
            #print('min_d - '+str(min_d), 'Lookin for the min days in the table')
    # Заполнение строки в артикуле с минимальным ocтатком в днях        
            if min_d == w[k][21] and group in w[k] and man_load < man_load_max:
                print(str(k),'man_load-'+str(man_load))
                print(str(k),"Минимальный остаток-" + str(min_d))
                # Считаем то, какое кол-во нужно добавить в план
                l_amount_min = kft_min(k, days)
                # Считаем остатки в днях, если бы мы добавили это кол-во в план
                l_residual_days = l_amount_min / w[k][13] + w[k][21]
                # Считаем загруженность производста, если бы мы добавили это кол-во в план
                man_load += l_amount_min*w[k][7]
                # Пишем проверку, которая сначала проверяет загруженность производства, потом остатки кроя
                if man_load <= man_load_max:
                    if l_amount_min <= w[k][23]:
                        print(k, 'Запонение ячейки для минимального остатка. Мощностей хватеает. Кроя хватает. ', l_amount_min)
                        print(k, 'man_load', man_load)
                        sew['Y'+str(k+1)].value = l_amount_min 
                    elif w[k][23]!= 0 and l_amount_min > w[k][23]:
                        man_load -= l_amount_min*w[k][7]
                        l_amount_min = w[k][23]
                        man_load += l_amount_min*w[k][7]
                        print(k,'Заполнение ячейки для минимального остатка. Мощностей хватает. Кроя не хватает', l_amount_min)
                        print(k, 'man_load', man_load)
                        sew['Y'+str(k+1)].value = l_amount_min
                    elif w[k][23] == 0 and l_amount_min > w[k][23]:
                        man_load -= l_amount_min*w[k][7]
                        l_amount_min = w[k][23]
                        man_load += l_amount_min*w[k][7]
                        print(k, 'Заполнение ячейки для минимального остатка. Мощностей хватает. Кроя нет', l_amount_min)
                        print(k, 'man_load', man_load)
                    else:
                        print(k, 'Boom! Минимальные остатки. Мощностей хватает. Меня не должно быть здесь.')
                elif man_load > man_load_max:
                    l_amount_min = int((l_amount_min - (man_load - man_load_max)/w[k][7])/w[k][32])*w[k][32]
                    print(k, 'Пересчитываем кол-во в плане из-за перегрузки мощностей', l_amount_min)
                    if l_amount_min <= w[k][23]:
                        print(k, 'Запонение ячейки для минимального остатка. Мощностей не хватеает. Кроя хватает', l_amount_min)
                        print(k, 'man_load', man_load)
                        sew['Y'+str(k+1)].value = l_amount_min 
                    elif w[k][23]> 0 and l_amount_min > w[k][23]:
                        l_amount_min = w[k][23]
                        print(k,'Заполнение ячейки для минимального остатка. Мощностей не хватает. Кроя не хватает', l_amount_min)
                        print(k, 'man_load', man_load)
                        sew['Y'+str(k+1)].value = l_amount_min
                    elif w[k][23] <= 0 and l_amount_min > w[k][23]:
                        l_amount_min = w[k][23]
                        print(k, 'Заполнение ячейки для минимального остатка. Мощностей не хватает. Кроя нет', l_amount_min)
                        print(k, 'man_load', man_load)
                    else:
                        print(k, 'Boom! Минимальные остатки. Мощностей не хватает. Меня не должно быть здесь.')
                else:
                    print(k, 'Boom! Минимальные остатки. Не прошел проверку на мощности. Меня не должно быть здесь.')
    # Заполнение планан для всего артикула ограничителей, в котором встретился минимальный остаток в днях
                for m in range(len(w)):
                    if w[m][6] == w[k][6] and w[m] != w[k]:
                        l_amount = kft(m, l_residual_days)
                        if l_amount > 0:
                            print(m, 'Проверка того, что остатки НЕдостаточно большие, что бы заполнять')
                            man_load += l_amount*w[m][7]
                            if man_load <= man_load_max:
                                if l_amount <= w[m][23]:
                                    print(m, 'Заполнение ячейки для НЕ минимального остатка. Мощностей хватает. Крой есть', l_amount)
                                    print(m, 'man_load', man_load)
                                    sew['Y'+str(m+1)].value = l_amount
                                elif w[m][23]>0 and l_amount > w[m][23]:
                                    man_load -= l_amount*w[m][7]
                                    l_amount = w[m][23]
                                    man_load += l_amount*w[m][7]
                                    print(m, 'Заполнение ячейки для НЕ минимального остатка. Мощностей хватает. Крой не хватает', l_amount)
                                    print(m, 'man_load', man_load)
                                    sew['Y'+str(m+1)].value = l_amount
                                elif w[m][23] <= 0 and l_amount > w[m][23]:
                                    man_load -= l_amount*w[m][7]
                                    l_amount = w[m][23]
                                    man_load += l_amount*w[m][7]
                                    print(m, 'Заполнение ячейки для НЕ минимального остатка. Мощностей хватает. Кроя нет', l_amount)
                                    print(m, 'man_load', man_load)
                                else:
                                    print(m, 'Boom! НЕ минимальные остатки. Мощностей хватает. Меня не должно быть здесь.')
                            elif man_load > man_load_max and count_numbers_of_man_load_cycles == 0:
                                print(m, 'count_numbers_of_man_load_cycles', count_numbers_of_man_load_cycles)
                                count_numbers_of_man_load_cycles += 1
                                print(m, 'l_amount_DO', l_amount)
                                print(m, 'man_load_DO', man_load)
                                l_amount = int((l_amount - (man_load - man_load_max)/w[m][7])/w[m][32])*w[m][32]
                                print(m, 'l_amount_POSLE', l_amount)
                                print(m, 'man_load_POSLE', man_load)
                                if l_amount > 0:
                                    print(m, 'Проверка того, что кол-во, добавляемое в план после загрузки про-ва положительное')
                                    if l_amount <= w[m][23]:
                                        print(m,'Заполнение ячейки для НЕ минимального остатка. Мощностей НЕ хватает. Крой есть', l_amount )
                                        print(m, 'man_load', man_load)
                                        sew['Y'+str(m+1)].value = l_amount
                                    elif w[m][23] > 0 and l_amount > w[m][23]:
                                        l_amount = w[m][23]
                                        print(k,'Заполнение ячейки для НЕ минимального остатка. Мощностей не хватает. Кроя не хватает', l_amount)
                                        print(m, 'man_load', man_load)
                                        sew['Y'+str(m+1)].value = l_amount
                                    elif w[m][23] <= 0 and l_amount > w[m][23]:
                                        l_amount = w[m][23]
                                        print(m,'Заполнение ячейки для НЕ минимального остатка. Мощностей не хватает. Нет кроя', l_amount)
                                        print(m, 'man_load', man_load)
                                    else:
                                        print(m, 'Boom! Не минимальные остатки. Мощностей не хватает. Меня не должно быть здесь.')
                                elif l_amount <= 0:
                                    print(m, 'Проверка того, что кол-во, добавляемое в план НЕ положительное')
                                else:
                                    print(m, 'Boom! Не минимальные остатки. Мощностей не хватает. Проверка НЕотрицтельности кол-ва в плане. Меня не должно быть здесь.')
                            elif count_numbers_of_man_load_cycles != 0:
                                print(m, 'Дошел до проверки count_numbers_of_man_load_cycles != 0', count_numbers_of_man_load_cycles)
                            else:
                                print(m, 'Boom! Не минимальные остатки. Не прошел проверку на мощности. Меня не должно быть здесь.')
                        elif l_amount <= 0:
                            print(m, 'Проверка того, что остатки достаточно большие, что бы заполнять')
                        else:
                            print(m, 'Boom! Проверка остатков. Меня не должно быть здесь')




    print(man_load)
    wb.save('C:\\Users\\d.maryanovskiy\\Desktop\\{} - БКГ пошив.xlsx'.format(current_date))
    return 'Done'

In [14]:
def plan_saving():
    plan = load_workbook('C:\\Users\\d.maryanovskiy\\Desktop\\{} - БКГ пошив.xlsx'.format(current_date))
    plan.create_sheet('Plan')
    pl = plan['Plan']
    sew = plan['Питон']
    for row in range(19, 699):
        if sew['Y'+str(row)].value != None:
            pl.append([sew['A'+str(row)].value, sew['B'+str(row)].value, sew['C'+str(row)].value, sew['D'+str(row)].value,
                      sew['I'+str(row)].value, sew['K'+str(row)].value, sew['J'+str(row)].value, sew['Y'+str(row)].value, '',
                      sew['H'+str(row)].value] )
    plan.active = pl
    plan.save('C:\\Users\\d.maryanovskiy\\Desktop\\{} - План БКГ пошив.xlsx'.format(current_date))

In [15]:
def delete_rows_and_cols():
    book = load_workbook('C:\\Users\\d.maryanovskiy\\Desktop\\{} - План БКГ пошив.xlsx'.format(current_date))
    sheet = book['Plan']
    group_list = []
    for i in sheet['E']:
        if i.value != None and i.value != 'Группа планирования':
            group_list.append(i.value)
            group_set = set(group_list)
    print(group_set)
    for group in group_set:
        wb = load_workbook('C:\\Users\\d.maryanovskiy\\Desktop\\{} - План БКГ пошив.xlsx'.format(current_date))
        ws = sheet_clearing(wb)
#    print([i for i in ws.values])
        row_number=2
        while row_number <= ws.max_row:
            if ws.cell(row=row_number, column=5).value != group:
                ws.delete_rows(row_number)
            else:
                row_number+=1
        delete_cols(ws)
        wb.save('C:\\Users\\d.maryanovskiy\\Desktop\\{} - План на {}.xlsx'.format(current_date, group))

def sheet_clearing(wb):
    sheet_list = wb.sheetnames
    b = len(sheet_list)
    for i in range(b-1):
        wb.active = 0
        wb.remove(wb.active)
    plan = wb.active
    return plan

def delete_cols(ws):
    ws.delete_cols(9, 20)
    ws.delete_cols(5, 2)
    ws.delete_cols(1, 3)
    return ws